In [5]:
import os
import logging
from logging import config
import openai
from dotenv import load_dotenv
import streamlit as st
from streamlit_chat import message
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

In [6]:
# Setting up logger
# log levels: DEBUG, INFO, WARNING, ERROR, CRITICAL

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [7]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [8]:
# Setting up the app
st.set_page_config(
    page_title="GPT-3 Chat",
    page_icon="🤖",
    layout="centered",
    initial_sidebar_state="auto",
)


In [3]:
def get_audio_from_text_and_save_to_file(api_key,
                                         text,
                                         speech_file_path,
                                         model="tts-1",
                                         voice="alloy"):
    client = openai.OpenAI(api_key=api_key)
    response = client.audio.speech.create(
        model=model,
        voice=voice,
        input=text
    )
    response.stream_to_file(speech_file_path)

In [9]:
my_voice_id = "bSePwwVOKgYwIz49RvoU"
my_api_key = os.getenv("ELEVENLABSKEY")

In [15]:
import requests

CHUNK_SIZE = 1024
url = f"https://api.elevenlabs.io/v1/text-to-speech/{my_voice_id}"

headers = {
    "Accept": "audio/mpeg",
    "Content-Type": "application/json",
    "xi-api-key": my_api_key
}

data = {
    "text": "Hallo Emilia, wie geht's dir heute morgen? Ich hole dich gleich aus dem Kindergarten ab",
                                       "model_id": "eleven_multilingual_v2",
    "voice_settings": {
        "stability": 0.5,
        "similarity_boost": 0.5
    }
}

response = requests.post(url, json=data, headers=headers)
with open('output.mp3', 'wb') as f:
    for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
        if chunk:
            f.write(chunk)

In [21]:
import requests
import json

url = "https://api.elevenlabs.io/v1/voices"

response = requests.request("GET", url)

print(json.dumps(response.json(), indent=4, sort_keys=True))

{
    "voices": [
        {
            "available_for_tiers": [],
            "category": "premade",
            "description": null,
            "fine_tuning": {
                "fine_tuning_requested": false,
                "finetuning_state": "not_started",
                "is_allowed_to_fine_tune": false,
                "language": null,
                "manual_verification": null,
                "manual_verification_requested": false,
                "slice_ids": null,
                "verification_attempts": null,
                "verification_attempts_count": 0,
                "verification_failures": []
            },
            "high_quality_base_model_ids": [],
            "labels": {
                "accent": "american",
                "age": "young",
                "description": "calm",
                "gender": "female",
                "use case": "narration"
            },
            "name": "Rachel",
            "preview_url": "https://storage.googleapis.com/e

In [26]:
for voice in response.json()["voices"]:
    # print name, category and labels
    print(f"{voice['name']} ({voice['category']}) ID: {voice['voice_id']} - {voice['labels']}")

# get voice_id by name
def get_voice_id_by_name(name):
    return [voice["voice_id"] for voice in response.json()["voices"] if voice["name"] == name][0]

get_voice_id_by_name("Rachel")
    

Rachel (premade) ID: 21m00Tcm4TlvDq8ikWAM - {'accent': 'american', 'description': 'calm', 'age': 'young', 'gender': 'female', 'use case': 'narration'}
Drew (premade) ID: 29vD33N1CtxCmqQRPOHJ - {'accent': 'american', 'description': 'well-rounded', 'age': 'middle aged', 'gender': 'male', 'use case': 'news'}
Clyde (premade) ID: 2EiwWnXFnvU5JabPnv8n - {'accent': 'american', 'description': 'war veteran', 'age': 'middle aged', 'gender': 'male', 'use case': 'video games'}
Paul (premade) ID: 5Q0t7uMcjvnagumLfvZi - {'accent': 'american', 'description': 'ground reporter', 'age': 'middle aged', 'gender': 'male', 'use case': 'news'}
Domi (premade) ID: AZnzlk1XvdvUeBnXmlld - {'accent': 'american', 'description': 'strong', 'age': 'young', 'gender': 'female', 'use case': 'narration'}
Dave (premade) ID: CYw3kZ02Hs0563khs1Fj - {'accent': 'british-essex', 'description': 'conversational', 'age': 'young', 'gender': 'male', 'use case': 'video games'}
Fin (premade) ID: D38z5RcWu1voky8WS1ja - {'accent': 'iri

'21m00Tcm4TlvDq8ikWAM'

In [27]:
from src import creds_db
print(creds_db.DBConfig().DB_HOST)

db-narraitive.cvx466dfq2il.eu-central-1.rds.amazonaws.com


In [32]:
from datetime import datetime
import psycopg2

def connect_to_db():
    DB_HOST = os.getenv('DB_HOST')
    DB_USER = os.getenv('DB_USER')
    DB_PASSWORD = os.getenv('DB_PASSWORD')
    DB_PORT = os.getenv('DB_PORT')
    DB_NAME = os.getenv('DB_NAME')
    return psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )


def insert_voice(name, voice_id, category, training_files, labels):
    creation_time = datetime.now()
    with connect_to_db() as conn:
        with conn.cursor() as cursor:
            cursor.execute("""
                INSERT INTO voices (name, voice_id, category, training_files, labels) 
                VALUES (%s, %s, %s, %s, %s)
            """, (name, voice_id, category, training_files, json.dumps(labels)))
            conn.commit()

for voice in response.json()["voices"]:
    # print name, category and labels
    insert_voice(voice["name"], voice["voice_id"], voice["category"], [], voice["labels"])


In [20]:
import requests

url = "https://api.elevenlabs.io/v1/models"

response = requests.request("GET", url)

print(json.dumps(response.json(), indent=4, sort_keys=True))

{
    "detail": {
        "message": "User not found, it is likely still being created.",
        "status": "user_not_found"
    }
}


In [16]:
from elevenlabs import clone, generate
from elevenlabs import set_api_key

set_api_key(os.getenv("ELEVENLABSKEY"))
voice = clone(
    name = "Faffi",
    files = ["../voice_input/Faffi_Leseprobe_1.m4a", "../voice_input/Faffi_Leseprobe_2.m4a", "../voice_input/Faffi_Leseprobe_3.m4a", "../voice_input/Faffi_Leseprobe_4.m4a", "../voice_input/Faffi_Leseprobe_5.m4a",]
)
voice_id = voice.voice_id
audio = generate("Hallo Emilia, wie geht's dir heute morgen? Ich hole dich gleich aus dem Kindergarten ab!", voice=voice, model="eleven_multilingual_v2")


In [18]:
voice_id

'rtMnJe9oyLFTEgYRg4kU'

In [19]:
voice

Voice(voice_id='rtMnJe9oyLFTEgYRg4kU', name='Faffi', category='cloned', description='', labels={}, samples=[VoiceSample(sample_id='SHTpQL3QnGGHIHRQQ7EZ', file_name='Faffi_Leseprobe_5_6364654480.mp3', mime_type='audio/mpeg', size_bytes=614835, hash='39800886aec179ece3c43e27883e34f8'), VoiceSample(sample_id='Wv75dhhaBl3sSaHbL9kT', file_name='Faffi_Leseprobe_4_6364655008.mp3', mime_type='audio/mpeg', size_bytes=428354, hash='09118752594681df07c1704440a053e9'), VoiceSample(sample_id='pWoKfAXfDD7W20xXEIHN', file_name='Faffi_Leseprobe_1_6364651488.mp3', mime_type='audio/mpeg', size_bytes=307281, hash='925bdaef4c88153fe9d62756235e72db'), VoiceSample(sample_id='uYQubwWSccIFt95Ux2AM', file_name='Faffi_Leseprobe_2_6364655536.mp3', mime_type='audio/mpeg', size_bytes=558123, hash='cf2fbd98d74e6db670682679604e39c9'), VoiceSample(sample_id='zyFWG1ht4N1R3ToVLhiI', file_name='Faffi_Leseprobe_3_6364655360.mp3', mime_type='audio/mpeg', size_bytes=559503, hash='b3fcc570259d92bd1a97800060721893')], design

In [17]:
from elevenlabs import play
play(audio)

In [ ]:
def get_audio_from_text_elevenlabs(api_key,
                        text,
                        model="tts-1",
                        voice="alloy"):
    client = openai.OpenAI(api_key=api_key)
    response = client.audio.speech.create(
        model=model,
        voice="alloy",
        input=text
    )
    return response

In [5]:
get_audio_from_text_and_save_to_file(os.getenv("OPENAI_API_KEY"),
                                     "Es war einmal ein Bär namens Bruno. Er lebte im tiefen Dschungel, umgeben von hohen Bäumen und bunten Blumen.",
                                     "test.mp3")

In [8]:
import base64
import requests

# OpenAI API Key
api_key = os.getenv("OPENAI_API_KEY")

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "../img/Emilia.png"

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

payload = {
    "model": "gpt-4-vision-preview",
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Please describe the character in this image in about 4-5 sentences and don't talk about the image itself. Just describe the character and start with 'A toddler, A boy.. / a girl / a man..."
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                    }
                }
            ]
        }
    ],
    "max_tokens": 300
}
for i_try in range(10):
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    if response.status_code == 200:
        break

print(response.json())

{'id': 'chatcmpl-8LG3fMhuFf8F7Nim7Dw4u89EEyn5n', 'object': 'chat.completion', 'created': 1700077307, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 816, 'completion_tokens': 101, 'total_tokens': 917}, 'choices': [{'message': {'role': 'assistant', 'content': "A toddler with light blonde hair stands in an open field, grasping a small object in their hand. They are clad in a cozy, blue zip-up jacket and patterned pants, with their feet fitted into pink-fringed boots, hinting at a playful sense of style. The child's expression is one of gentle curiosity, a common characteristic of their explorative age. Their focus seems directed at their surroundings, possibly taking in the vastness of the environment or looking at someone or something off-camera."}, 'finish_details': {'type': 'stop', 'stop': '<|fim_suffix|>'}, 'index': 0}]}


In [7]:
response.status_code

200

In [15]:
import base64
import requests

# OpenAI API Key
api_key = os.getenv("OPENAI_API_KEY")

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "../img/Thorin.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

payload = {
    "model": "gpt-4-vision-preview",
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Please describe the character in this image in about 2 sentences and don't talk about the image itself. Just describe the character and start with 'A toddler, A boy.. / a girl / a man..."
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                    }
                }
            ]
        }
    ],
    "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

TypeError: 'Response' object is not subscriptable

In [20]:
response.json()["choices"][0]['message']['content']

'A toddler with blonde hair and blue eyes stands looking into the camera. They are wearing a cozy blue sweater adorned with a cute animal design.'